In [10]:
import pandas as pd
import numpy as np
import os

In [11]:
def move_column(df, cn, cp):
    # 定义移动的列
    m_column = df.pop(cn)
    # 关键代码(位置, 列名, 要移动的列) 
    df.insert(cp, cn, m_column)
    return df
    # 打印提示信息
    print('列位置更改成功')

# # 调用方法
# move_column(file_move, column_name, column_position)

In [12]:
# 读txt文件
def readPatList(pat_file_list):
    ret_list_list=[]
    for i in pat_file_list:
        patlist = pd.read_table(i,sep='\t',header=None,index_col=None)[0].tolist()
        ret_list_list.append(patlist)
    
    ret_list =[]
    for i in ret_list_list:
        for j in i:
            ret_list.append(j)
    
    ret_list_ret = list(set(ret_list))
    
    print(f'读取 {len(pat_file_list)} 个文件,重合 {len(ret_list)-len(ret_list_ret)},共 {len(ret_list_ret)} 个病人')
    return ret_list_ret

# readPatList([r'../../分组/jsph_gbm.txt',r'../../分组/jsph_lgg.txt',
#                                      r'../../分组/TEST_jsph_gbm.txt',r'../../分组/TEST_jsph_lgg.txt'])

# 获取CIBERSORT和radiomics数据的交叉

In [13]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  打开影像组学文件
df_radiomics = pd.read_table('20_RiskScore_radiomics1402_pretreated_OS.csv',sep=',',index_col=0)
feature_not_radiomics = df_radiomics.columns.tolist()[:3]
print(feature_not_radiomics)
df_radiomics

['status', 'time', 'RiskScore_h0']


,status,time,RiskScore_h0,log.sigma.5.mm.3D_glcm_Idn,squareroot_firstorder_Median,lbp.3D.k_glrlm_ShortRunEmphasis,wavelet.LLH_glszm_GrayLevelNonUniformity,log.sigma.5.mm.3D_firstorder_90Percentile,wavelet.LHH_ngtdm_Contrast,square_glrlm_RunEntropy,...,original_shape_MinorAxisLength,wavelet.LLH_firstorder_Mean,original_shape_Sphericity,wavelet.LLL_glcm_Idmn,wavelet.LHL_gldm_DependenceVariance,wavelet.HLL_gldm_DependenceVariance,log.sigma.3.mm.3D_glszm_ZoneEntropy,log.sigma.5.mm.3D_firstorder_Median,lbp.2D_firstorder_InterquartileRange,wavelet.LHH_gldm_DependenceNonUniformityNormalized
X0907236_ZhouJiSheng,1,6.83,0.896298,0.978929,418.103580,0.419132,73.934508,13.716698,0.001638,4.957925,...,56.688722,-1.006697,0.498970,0.998059,29.992383,33.763932,6.205362,-24.819478,2.5,0.104869
X0875917_LiYan,1,15.70,0.901296,0.976438,469.206017,0.372534,48.381674,18.177860,0.000817,4.973058,...,59.609625,-0.910070,0.662562,0.998251,33.161329,33.761676,6.552695,-29.572262,2.0,0.078167
TCGA-27-1830,1,5.13,2.621098,0.976809,361.326537,0.392058,91.859655,-1.832864,0.003141,4.811214,...,33.948305,-2.664174,0.616121,0.997026,28.714363,30.516095,5.870566,-48.545515,2.0,0.080178
TCGA-DU-5854,0,8.57,0.627098,0.975821,378.441206,0.297715,13.598095,9.866241,0.001674,4.827622,...,42.470084,-0.894696,0.663250,0.997674,27.613160,34.625264,5.097847,-20.290770,2.0,0.080079
HF1588,0,74.20,1.181956,0.968646,389.782128,0.409873,52.251438,12.997869,0.019593,4.761555,...,37.550941,-0.326498,0.531358,0.996102,25.230655,27.546266,5.379736,-40.516292,3.0,0.078900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X1052598_ShiHongMei,0,29.67,0.619839,0.973971,545.344729,0.405578,66.154018,-6.453781,0.001158,5.066364,...,51.721233,-1.941097,0.682478,0.998560,30.617479,34.898009,6.441233,-57.654155,3.0,0.085675
X0930877_XuPingLan,1,8.63,1.547634,0.980352,496.023289,0.296317,31.711602,19.271812,0.010592,5.255231,...,53.134315,-0.414315,0.613724,0.998826,33.895249,34.235136,6.592257,-24.207621,2.0,0.081979
TCGA-DU-6395,1,49.70,2.314271,0.977762,416.562123,0.339940,437.999479,4.845201,0.002410,4.611289,...,54.899323,-1.953735,0.373961,0.996877,33.189645,39.412890,4.749348,-26.371862,2.0,0.072793
HF1587,0,74.20,1.226855,0.972784,447.538339,0.349452,85.220925,6.252650,0.001351,4.968237,...,41.525773,-1.323163,0.691430,0.997400,30.526584,34.167278,5.717379,-53.489113,2.0,0.079584


In [14]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  打开CIBERSORT结果文件
df_cibersort = pd.read_table(r'../1.CIBERSORT本地计算/TcgaRbrtJsph_cibersort.csv',sep=',',index_col=0)
move_column(df_cibersort, 'Macrophages M2', 0)
move_column(df_cibersort, 'Macrophages M1', 0)
move_column(df_cibersort, 'Macrophages M0', 0)

# 新增一列Macrophages 
MacrophageAll_df_orig = df_cibersort[['Macrophages M0','Macrophages M1','Macrophages M2']]
MacrophageAll_list = []
for row in MacrophageAll_df_orig.index:
    sum = 0
    for col in MacrophageAll_df_orig.columns:
        sum += MacrophageAll_df_orig.loc[row,col]
    MacrophageAll_list.append(sum)
    
df_cibersort.insert(3, 'Macrophages', MacrophageAll_list)  # 新列

p_threshold = 0.05  # 筛选p值
df_cibersort = df_cibersort[df_cibersort["P-value"] <= p_threshold]

feature_not_cibersort = df_cibersort.columns.tolist()[-3:]
print(feature_not_cibersort)
df_cibersort

['P-value', 'Correlation', 'RMSE']


,Macrophages M0,Macrophages M1,Macrophages M2,Macrophages,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,...,Monocytes,Dendritic cells resting,Dendritic cells activated,Mast cells resting,Mast cells activated,Eosinophils,Neutrophils,P-value,Correlation,RMSE
X1002249_GuCongXiang,0.000000,0.010115,0.472494,0.482609,0.000000,0.013321,0.028396,0.048041,0.000000,0.034102,...,0.243043,0.000000,0.007005,0.00000,0.067758,0.006636,0.000000,0.04,0.149711,1.054281
X1029842_XuLongSheng,0.000000,0.002645,0.451138,0.453783,0.000000,0.004771,0.051071,0.119777,0.000000,0.000000,...,0.081126,0.000000,0.029672,0.08935,0.000000,0.000000,0.007439,0.04,0.133902,1.038183
X1164833_DongXiangZheng,0.274952,0.010602,0.382194,0.667748,0.000000,0.002037,0.002735,0.025598,0.000000,0.026502,...,0.011923,0.000000,0.001914,0.00000,0.123865,0.002685,0.000000,0.01,0.182948,1.059803
X1163024_XuMeiFang,0.178924,0.029860,0.312096,0.520880,0.046989,0.000000,0.000000,0.053835,0.000000,0.104998,...,0.049842,0.000000,0.000000,0.00000,0.161106,0.003130,0.000000,0.00,0.333022,0.958238
TCGA-06-2559,0.312608,0.016152,0.353047,0.681806,0.000000,0.015852,0.005754,0.022076,0.000000,0.120807,...,0.002422,0.000000,0.000000,0.00000,0.025756,0.000000,0.019448,0.02,0.162790,1.051020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MD621233,0.000000,0.012909,0.138133,0.151043,0.080971,0.000000,0.000000,0.203351,0.021786,0.000000,...,0.000000,0.076894,0.000000,0.00000,0.275938,0.000000,0.000000,0.03,0.176341,1.040316
NT4,0.000000,0.008006,0.202743,0.210749,0.070798,0.000000,0.000000,0.179094,0.000000,0.000000,...,0.008488,0.089075,0.000000,0.00000,0.086813,0.000000,0.180117,0.02,0.243004,0.991865
NT5,0.000000,0.030730,0.056159,0.086889,0.028319,0.000000,0.000000,0.237129,0.000000,0.000000,...,0.000000,0.133045,0.000000,0.00000,0.229040,0.000000,0.045461,0.05,0.148217,1.050252
NT6,0.000000,0.012482,0.177674,0.190156,0.055181,0.000000,0.000000,0.187551,0.000000,0.000000,...,0.103842,0.092328,0.000000,0.00000,0.124618,0.001697,0.092690,0.02,0.241654,0.988483


In [15]:
# ━━━━━━━━━━━━━━━━━━━━━━━━  获取交叉
df_cibersort_list = df_cibersort.index.tolist()
df_radiomics_list = df_radiomics.index.tolist()
patCrossCondition = list(set(df_radiomics_list) & set(df_cibersort_list))
print(f'有{len(patCrossCondition)}个交叉')

df_tmp = pd.concat([df_cibersort,df_radiomics],axis=1).T[patCrossCondition].T
df_tmp

有103个交叉


,Macrophages M0,Macrophages M1,Macrophages M2,Macrophages,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,...,original_shape_MinorAxisLength,wavelet.LLH_firstorder_Mean,original_shape_Sphericity,wavelet.LLL_glcm_Idmn,wavelet.LHL_gldm_DependenceVariance,wavelet.HLL_gldm_DependenceVariance,log.sigma.3.mm.3D_glszm_ZoneEntropy,log.sigma.5.mm.3D_firstorder_Median,lbp.2D_firstorder_InterquartileRange,wavelet.LHH_gldm_DependenceNonUniformityNormalized
TCGA-FG-6691,0.000000,0.000000,0.248227,0.248227,0.001007,0.001302,0.000521,0.000000,0.0,0.139405,...,49.450908,-1.704192,0.658855,0.996149,31.233171,36.202846,6.025922,-68.106323,2.0,0.080552
X1163024_XuMeiFang,0.178924,0.029860,0.312096,0.520880,0.046989,0.000000,0.000000,0.053835,0.0,0.104998,...,63.973382,-1.332981,0.474454,0.998524,32.137390,35.174766,6.736800,-35.300869,2.0,0.077682
HF1318,0.069575,0.046622,0.246821,0.363018,0.072369,0.000000,0.000000,0.126710,0.0,0.099128,...,49.808504,-0.660026,0.661939,0.996765,31.588563,30.319171,6.116497,-34.393970,2.0,0.080098
TCGA-DU-6395,0.000000,0.001788,0.056608,0.058396,0.007896,0.000000,0.002533,0.006441,0.0,0.140620,...,54.899323,-1.953735,0.373961,0.996877,33.189645,39.412890,4.749348,-26.371862,2.0,0.072793
TCGA-HT-7478,0.000000,0.014363,0.280610,0.294974,0.002638,0.000138,0.000584,0.036085,0.0,0.103042,...,24.643490,-4.917792,0.744970,0.996082,40.247498,39.488746,4.891224,-75.415916,2.0,0.058870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HF1058,0.000000,0.000000,0.042107,0.042107,0.000000,0.109003,0.003388,0.072139,0.0,0.065274,...,46.990627,-2.476793,0.722270,0.997609,32.425907,33.608647,6.247167,-40.503073,2.0,0.081454
TCGA-HT-7854,0.000000,0.023155,0.268518,0.291673,0.020603,0.000000,0.000000,0.001658,0.0,0.051258,...,40.486537,-1.216840,0.716435,0.997860,26.904390,29.361962,5.568802,-30.472067,3.0,0.094837
TCGA-HT-7602,0.014295,0.036539,0.076050,0.126884,0.002490,0.022242,0.000000,0.052210,0.0,0.116101,...,49.028322,-0.622467,0.543791,0.998275,31.552612,35.635830,5.756778,-20.974491,2.0,0.085408
HF1325,0.000000,0.000000,0.138087,0.138087,0.080497,0.000000,0.000000,0.099833,0.0,0.099404,...,48.204353,-1.829921,0.684675,0.996511,24.718670,31.358952,6.047149,-70.295633,2.0,0.085955


In [16]:
# 要把 数据列 变成分类的 特征名们
features_all = list(set(df_tmp.columns.tolist()) - (set(feature_not_radiomics) | set(feature_not_cibersort)))
print(f'【{df_tmp.shape[0]}行数据】*【{len(features_all)}】=【{df_tmp.shape[0]*len(features_all)}行数据】')

# 开始处理
df_list = []
reserveList=list(set(feature_not_radiomics) | set(feature_not_cibersort))

for i in features_all:
    df_single = df_tmp[reserveList]  # 原来的列
    new_col = df_tmp[i].values.tolist()  # 新列

    # 在第一列加入新的
    df_single.insert(0, 'enrichment_value', new_col)
    # 把分类重复n次之后加入df中
    class_col = [i for j in range(df_tmp.shape[0])]
    df_single.insert(0, 'class', class_col)

    df_list.append(df_single)
    
df_list[0]

【103行数据】*【43】=【4429行数据】


,class,enrichment_value,RiskScore_h0,time,RMSE,Correlation,P-value,status
TCGA-FG-6691,T cells CD4 memory activated,0.0,0.253208,41.90,0.986758,0.270245,0.00,0.0
X1163024_XuMeiFang,T cells CD4 memory activated,0.0,2.481322,16.37,0.958238,0.333022,0.00,1.0
HF1318,T cells CD4 memory activated,0.0,0.860631,7.60,1.027758,0.151105,0.05,1.0
TCGA-DU-6395,T cells CD4 memory activated,0.0,2.314271,49.70,0.962302,0.388910,0.00,1.0
TCGA-HT-7478,T cells CD4 memory activated,0.0,0.109955,6.47,0.958867,0.322398,0.00,0.0
...,...,...,...,...,...,...,...,...
HF1058,T cells CD4 memory activated,0.0,1.069856,17.80,0.988031,0.295389,0.01,1.0
TCGA-HT-7854,T cells CD4 memory activated,0.0,0.871490,40.03,1.013979,0.219866,0.01,0.0
TCGA-HT-7602,T cells CD4 memory activated,0.0,0.845872,30.27,1.014844,0.173345,0.02,0.0
HF1325,T cells CD4 memory activated,0.0,0.371803,83.30,0.975827,0.287069,0.01,0.0


In [17]:
# 把散装的合并起来
df_long = pd.concat(df_list,axis =0,ignore_index=False)
df_long

df_long.to_csv('→长数据_供绘散点图.csv',index=True,header=True)

# 重构（以下废弃）

In [9]:
# # 打开文件位置
# jsph_lgg = readPatList([r'../../分组/jsph_lgg.txt'])
# jsph_gbm = readPatList([r'../../分组/jsph_gbm.txt'])
# TEST_jsph_lgg = readPatList([r'../../分组/TEST_jsph_lgg.txt'])
# TEST_jsph_gbm = readPatList([r'../../分组/TEST_jsph_gbm.txt'])
# jsph = readPatList([r'../../分组/jsph_lgg.txt',r'../../分组/jsph_gbm.txt',
#                    r'../../分组/TEST_jsph_lgg.txt',r'../../分组/TEST_jsph_gbm.txt'])

# rbrt_lgg = readPatList([r'../../分组/rbrt_lgg.txt'])
# rbrt_gbm = readPatList([r'../../分组/rbrt_gbm.txt'])
# rbrt = readPatList([r'../../分组/rbrt_lgg.txt',r'../../分组/rbrt_gbm.txt'])

# tcga_lgg = readPatList([r'../../分组/tcga_lgg.txt'])
# tcga_gbm = readPatList([r'../../分组/tcga_gbm.txt'])
# tcga = readPatList([r'../../分组/tcga_lgg.txt',r'../../分组/tcga_gbm.txt'])

In [10]:
# def main(file_path,save_name):
#     print(f'处理【{save_name}】')
#     # 读取病人列表
#     pat_list = readPatList(file_path)
#     # 取交集
#     pat_cross = list(set(pat_list) & set(df_tmp.index.tolist()))
    
#     df_long_matched = df_long.T[pat_cross].T
#     print(f'交集{len(df_mid)}人')
    
# #     save_name = r'../→CIBERSORT/→'+save_name+'_matched.csv'
# #     df_mid.to_csv(save_name,index=True,header=True)
    

In [11]:
# main([r'../../分组/tcga_lgg.txt'],'tcga_lgg')